# cifar110-PN

- slide: http://www.ms.k.u-tokyo.ac.jp/~gang/paper/niu_ibisml030.pdf
- ipynb: https://github.com/JihongJu/deep-pu-learning/blob/master/experiments/cifar/cifar110-PN.ipynb

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#CIFAR110" data-toc-modified-id="CIFAR110-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>CIFAR110</a></span><ul class="toc-item"><li><span><a href="#Load-Dataset" data-toc-modified-id="Load-Dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Dataset</a></span></li><li><span><a href="#Normalize" data-toc-modified-id="Normalize-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Normalize</a></span><ul class="toc-item"><li><span><a href="#[Def]-normalize" data-toc-modified-id="[Def]-normalize-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>[Def] normalize</a></span></li><li><span><a href="#exe" data-toc-modified-id="exe-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>exe</a></span></li></ul></li><li><span><a href="#Percentage-of-Missing-data" data-toc-modified-id="Percentage-of-Missing-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Percentage of Missing data</a></span><ul class="toc-item"><li><span><a href="#[Def]-synthesize_pu_labels" data-toc-modified-id="[Def]-synthesize_pu_labels-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>[Def] synthesize_pu_labels</a></span></li><li><span><a href="#[Def]-synthesize_pn_labels" data-toc-modified-id="[Def]-synthesize_pn_labels-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>[Def] synthesize_pn_labels</a></span></li><li><span><a href="#exe" data-toc-modified-id="exe-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>exe</a></span></li></ul></li></ul></li></ul></div>

## Import

In [1]:
from __future__ import (print_function, absolute_import)

import os
#-------------------------------------------------------------------------------
#  OPTIONAL
#  set a GPU viewed by TensorFlow
##---------------------------------------
##  - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
##---------------------------------------

import re

import numpy as np
print("numpy ver: {}".format(np.__version__))

import pandas as pd
print("pandas ver: {}".format(pd.__version__))

import keras
print("keras ver: {}".format(keras.__version__))
import keras.backend as K
from keras import datasets
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import (
    ReduceLROnPlateau,
    LearningRateScheduler,
    CSVLogger,
    EarlyStopping,
    ModelCheckpoint)

import tensorflow as tf
print("tensorflow ver: {}".format(tf.__version__))
# You can double check that you have the correct devices visible to TF
#   - https://stackoverflow.com/questions/37893755/tensorflow-set-cuda-visible-devices-within-jupyter
from tensorflow.python.client import device_lib
print("""
________________________________________
Visible GPUs from TensorFlow
________________________________________""")
for _device in device_lib.list_local_devices():
    match = re.search(pattern=r'name: "/device:(?P<name>[A-Z]{3}):(?P<device_num>\d{1})*',
                      string=str(_device))
    if match is None:
        print("Not Match")
        continue
    if match.group("name") == "CPU":
        name, device_num = match.group("name", "device_num")
        print()
        print("({}:{})".format(name, device_num))
        continue
    name, device_num = match.group("name", "device_num")
    match = re.search(pattern=r'.*pci bus id: (?P<pci_bus_id>\d{4}:\d{2}:\d{2}.\d{1}).*',
                      string=str(_device))
    if match is None:
        print("No GPUs")
        continue
    print("({}:{}: pci_bus_id: {})".format(name, device_num, match.group("pci_bus_id")))
print("________________________________________")

global _SESSION
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True
_SESSION = tf.Session(config=config)
K.set_session(_SESSION)


import keras_tqdm
from keras_tqdm import TQDMNotebookCallback
#print("keras_tqdm ver: {}".format(keras_tqdm.__version__))

numpy ver: 1.15.0
pandas ver: 0.23.3


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


keras ver: 2.2.2
tensorflow ver: 1.9.0

________________________________________
Visible GPUs from TensorFlow
________________________________________

(CPU:0)
(GPU:0: pci_bus_id: 0000:01:00.0)
________________________________________


## CIFAR110

In [2]:
num_classes = 11
batch_size = 128
epochs = 200
data_augmentation = True
checkpoint = None
# checkpoint = 'model_checkpoint_cifar10_wide_resnet.h5'
title = 'cifar110_vgg8'
# title = 'cifar110_wide_resnet'

### Load Dataset

In [3]:
(x_train_10, y_train_10), (x_test_10, y_test_10) = datasets.cifar10.load_data()
(x_train_100, y_train_100), (x_test_100, y_test_100) = datasets.cifar100.load_data()

y_train_10 = y_train_10 + 1
y_test_10 = y_test_10 + 1
y_train_100[...] = 0
y_test_100[...] = 0

In [4]:
x_train = np.concatenate((x_train_10, x_train_100), axis=0).astype('float32')
y_train = np.concatenate((y_train_10, y_train_100), axis=0).astype('int8')
x_test = np.concatenate((x_test_10, x_test_100), axis=0).astype('float32')
y_test = np.concatenate((y_test_10, y_test_100), axis=0).astype('int8')

x_train = x_train.astype(K.floatx())
x_test = x_test.astype(K.floatx())

# one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Normalize

#### [Def] normalize

In [5]:
def normalize(x, mean, std):
    """
    各チャネルの平均・標準偏差を用いて標準化
    """
    import copy
    x = copy.deepcopy(x)

    x -= np.array(object=mean, dtype=K.floatx())
    x /= np.array(object=std, dtype=K.floatx())
    return x

#### exe

In [6]:
print(x_train.shape)

mean = x_train.mean(axis=(0, 1, 2))
print(mean)
std = x_train.std(axis=(0, 1, 2))
print(std)

x_train = normalize(x=x_train, mean=mean, std=std)
x_test = normalize(x=x_test, mean=mean, std=std)

(100000, 32, 32, 3)
[127.3043  123.50989 113.14998]
[65.66327 63.76426 68.59032]


In [7]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(100000, 32, 32, 3) (100000, 11)
(20000, 32, 32, 3) (20000, 11)


### Percentage of Missing data

#### [Def] synthesize_pu_labels

In [8]:
def synthesize_pu_labels(Y, pct_missings=None, random_state=None,
                         verbose=False):
    """
    Synthesize PU labels.
    reference:
      - https://github.com/JihongJu/deep-pu-learning/blob/master/pulearn/utils.py
      - https://github.com/JihongJu/deep-pu-learning/blob/d896ca838e9629d9d418fd36f1138b9084c97bea/pulearn/utils.py
    """
    # To PU
    if pct_missings is None:
        pct_missings = np.arange(0., 1 + 1e-8, 0.1)
    Y_pu = {}
    np.random.seed(random_state)
    n_samples = len(Y)
    for pct in pct_missings:
        y = np.argmax(Y, 1)
        flip = np.random.rand(n_samples)
        y[(y != 0) & (flip < pct)] = 0
        Y_pu[pct] = np.eye(Y.shape[1])[y]
        if verbose is True:
            print('Positive (pct_missing={}):'.format(pct),
                  np.sum(np.argmax(Y, 1)>0), ' -> ', np.sum(y>0))
    return Y_pu

#### [Def] synthesize_pn_labels

In [9]:
def synthesize_pn_labels(Y, pct_missings=None, random_state=None,
                         verbose=False):
    """
    Synthesize PN labels.
    reference:
      - https://github.com/JihongJu/deep-pu-learning/blob/master/pulearn/utils.py
      - https://github.com/JihongJu/deep-pu-learning/blob/d896ca838e9629d9d418fd36f1138b9084c97bea/pulearn/utils.py
    """
    # To PN
    if pct_missings is None:
        pct_missings = np.arange(0., 1 + 1e-8, 0.1)
    idx = {}
    np.random.seed(random_state)
    n_samples = len(Y)
    for pct in pct_missings:
        y = np.argmax(Y, 1)
        flip = np.random.rand(n_samples)
        mask = flip < pct
        y[mask] = -1
        idx[pct] = np.where(np.invert(mask))[0]
        if verbose is True:
            print('Positive vs. Negative (pct={}):'.format(pct),
                  np.sum(y > 0), ' vs.', np.sum(y == 0))
    return idx

#### exe

In [20]:
pct_missing = 0.2
y_train_pu = synthesize_pu_labels(y_train, random_state=42, verbose=True)
y_train_pn = synthesize_pn_labels(y_train, random_state=42, verbose=True)
x_train_enc = x_train[y_train_pn[pct_missing],...]
y_train_enc = y_train[y_train_pn[pct_missing],...]
print(x_train_enc.shape, y_train_enc.shape)

Positive (pct_missing=0.0): 50000  ->  50000
Positive (pct_missing=0.1): 50000  ->  44990
Positive (pct_missing=0.2): 50000  ->  39992
Positive (pct_missing=0.30000000000000004): 50000  ->  34902
Positive (pct_missing=0.4): 50000  ->  30043
Positive (pct_missing=0.5): 50000  ->  24968
Positive (pct_missing=0.6000000000000001): 50000  ->  19937
Positive (pct_missing=0.7000000000000001): 50000  ->  14896
Positive (pct_missing=0.8): 50000  ->  10153
Positive (pct_missing=0.9): 50000  ->  4902
Positive (pct_missing=1.0): 50000  ->  0
Positive vs. Negative (pct=0.0): 50000  vs. 50000
Positive vs. Negative (pct=0.1): 44990  vs. 45014
Positive vs. Negative (pct=0.2): 39992  vs. 40104
Positive vs. Negative (pct=0.30000000000000004): 34902  vs. 34988
Positive vs. Negative (pct=0.4): 30043  vs. 30178
Positive vs. Negative (pct=0.5): 24968  vs. 24841
Positive vs. Negative (pct=0.6000000000000001): 19937  vs. 20106
Positive vs. Negative (pct=0.7000000000000001): 14896  vs. 15058
Positive vs. Negat

In [28]:
from sklearn.utils import class_weight

print(y_train_enc.shape)
y_train_dec = 1 - y_train_enc[:, 0]  # unlabel(クラスラベル:0) の one-hot encoding 表記を反転
print(y_train_dec.shape)
class_weights = class_weight.compute_class_weight('balanced',
                                                 [0, 1],
                                                 y_train_dec)

(80096, 11)
(80096,)


In [43]:
unlabel, label = np.bincount(y_train_dec.astype("int64"))
print("""\
negative = unlabeled (0) : {}
positive = labeled   (1) : {}""".format( unlabel, label) )

negative = unlabeled (0) : 40104
positive = labeled   (1) : 39992


In [25]:
class_weights

array([0.99860363, 1.00140028])

In [12]:
# class_weights = [1., 2.]
class_weight = {}
class_weight[0] = class_weights[0]
for k in range(1, num_classes):
    class_weight[k] = class_weights[1]


print('class_weight', class_weight)

class_weight {0: 0.9986036305605426, 1: 1.0014002800560111, 2: 1.0014002800560111, 3: 1.0014002800560111, 4: 1.0014002800560111, 5: 1.0014002800560111, 6: 1.0014002800560111, 7: 1.0014002800560111, 8: 1.0014002800560111, 9: 1.0014002800560111, 10: 1.0014002800560111}


In [13]:
def VGG8(input_shape, num_classes, t_vec=None):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
    from keras.layers import Conv2D, MaxPooling2D
    from keras.layers.advanced_activations import LeakyReLU

    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal',
                                input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer='he_normal'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(512, kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, kernel_initializer='he_normal'))
    # output translate (tensorflow only)
    if t_vec is not None:
        def translate(x):
            return x - t_vec
        model.add(Lambda(lambda x: translate(x)))
    model.add(Activation('softmax'))

    return model

In [14]:
if title == 'cifar110_vgg8':
    model = VGG8(input_shape=x_train_enc.shape[1:], num_classes=num_classes)
elif title == 'cifar110_wide_resnet':
    model = WideResidualNetwork(depth=28, width=8, dropout_rate=0.5,
                                classes=num_classes, include_top=True,
                                weights=None)
if checkpoint is not None:
    model = load_model(checkpoint)
    

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [15]:
# Checkpoint
checkpointer = ModelCheckpoint(
    filepath="model_checkpoint_{}_pn_{}.h5".format(title, pct_missing),
    verbose=1,
    save_best_only=True)

# csvlogger
csv_logger = CSVLogger(
    'csv_logger_{}_pn_{}.csv'.format(title, pct_missing))

# EarlyStopping
early_stopper = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=50)
# Reduce lr with schedule
# def schedule(epoch):
#     lr = K.get_value(model.optimizer.lr)
#     if epoch in [60, 120, 160]:
#         lr = lr * np.sqrt(0.1)
#     return lr
# lr_scheduler = LearningRateScheduler(schedule)

# Reduce lr on plateau
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=20,
                               min_lr=0.3e-6)

In [19]:
print('Training {} samples'.format(x_train_enc.shape[0]))

if not data_augmentation:
    print('No data augmentation applied.')
    model.fit(x_train_enc, y_train_enc,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=[csv_logger, checkpointer, early_stopper]) 
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        # randomly rotate images in the range (degrees, 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train_enc, y_train_enc,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        class_weight=class_weight,
                        callbacks=[csv_logger, checkpointer, early_stopper,
                                   lr_reducer, TQDMNotebookCallback()])
    model.save('{}_pn_{}.h5'.format(title, pct_missing))
    
    # Predict
    y_train_pred_enc = model.predict(x_train)
    y_test_pred_enc = model.predict(x_test)
    
    lst = []
    y_train_pred = np.argmax(y_train_pred_enc, axis=-1)
    y_train_true = np.argmax(y_train, axis=-1)
    y_train_label = np.argmax(y_train_pu[pct_missing], axis=-1)
    for y_pred, y_true, y_label in zip(y_train_pred, y_train_true, y_train_label):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_label, split='train'))

    y_test_pred = np.argmax(y_test_pred_enc, axis=-1)
    y_test_true = np.argmax(y_test, axis=-1)
    for y_pred, y_true in zip(y_test_pred, y_test_true):
        lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_true, split='test'))
    
    df = pd.DataFrame(lst)
    df.to_csv('prediction_{}_pn_{}.csv'.format(title, pct_missing), index=False, mode='a')

Training 80096 samples
Using real-time data augmentation.


Epoch 1/200


781/781 [==============================] - 28s 36ms/step - loss: 1.4905 - acc: 0.5349 - val_loss: 1.2444 - val_acc: 0.5904

Epoch 00001: val_loss improved from inf to 1.24440, saving model to model_checkpoint_cifar110_vgg8_pn_0.2.h5
Epoch 2/200


781/781 [==============================] - 27s 34ms/step - loss: 1.1971 - acc: 0.6001 - val_loss: 0.9938 - val_acc: 0.6654

Epoch 00002: val_loss improved from 1.24440 to 0.99378, saving model to model_checkpoint_cifar110_vgg8_pn_0.2.h5
Epoch 3/200


781/781 [==============================] - 27s 35ms/step - loss: 1.0823 - acc: 0.6332 - val_loss: 0.9261 - val_acc: 0.6867

Epoch 00003: val_loss improved from 0.99378 to 0.92607, saving model to model_checkpoint_cifar110_vgg8_pn_0.2.h5
Epoch 4/200


...

781/781 [==============================] - 25s 32ms/step - loss: 0.4647 - acc: 0.8403 - val_loss: 0.4530 - val_acc: 0.8518

Epoch 00196: val_loss did not improve from 0.43768
Epoch 197/200


781/781 [==============================] - 25s 32ms/step - loss: 0.4632 - acc: 0.8407 - val_loss: 0.4507 - val_acc: 0.8522

Epoch 00197: val_loss did not improve from 0.43768
Epoch 198/200


781/781 [==============================] - 25s 33ms/step - loss: 0.4696 - acc: 0.8390 - val_loss: 0.4648 - val_acc: 0.8477

Epoch 00198: val_loss did not improve from 0.43768
Epoch 199/200


781/781 [==============================] - 25s 32ms/step - loss: 0.4609 - acc: 0.8408 - val_loss: 0.4436 - val_acc: 0.8527

Epoch 00199: val_loss did not improve from 0.43768
Epoch 200/200


781/781 [==============================] - 25s 32ms/step - loss: 0.4626 - acc: 0.8414 - val_loss: 0.4606 - val_acc: 0.8506

Epoch 00200: val_loss did not improve from 0.43768



In [ ]:
model = load_model(filepath="model_checkpoint_cifar110_vgg8_pn_0.2.h5")

In [ ]:
# Predict
y_train_pred_enc = model.predict(x_train_normalized)
y_test_pred_enc = model.predict(x_test)

lst = []
y_train_pred = np.argmax(y_train_pred_enc, axis=-1)
y_train_true = np.argmax(y_train, axis=-1)
y_train_label = np.argmax(y_train_pu[pct_missing], axis=-1)
for y_pred, y_true, y_label in zip(y_train_pred, y_train_true, y_train_label):
    lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_label, split='train'))

y_test_pred = np.argmax(y_test_pred_enc, axis=-1)
y_test_true = np.argmax(y_test, axis=-1)
for y_pred, y_true in zip(y_test_pred, y_test_true):
    lst.append(dict(y_pred=y_pred, y_true=y_true, y_label=y_true, split='test'))

df = pd.DataFrame(lst)
df.to_csv('prediction_{}_pn_{}.csv'.format(title, pct_missing), index=False, mode='a')

In [ ]:
df